<a href="https://colab.research.google.com/github/Preksha1404/Retrieval-Augmented-Generation/blob/main/RAG_Application_Using_LangChain_Gemini_and_Weaviate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install weaviate-client langchain langchain_community langchain_huggingface tiktoken pypdf rapidocr-onnxruntime

In [2]:
from google.colab import userdata
weaviate_url = userdata.get('WEAVIATE_URL')
weaviate_api_key = userdata.get('WEAVIATE_API_KEY')

In [3]:
import os
import weaviate
from weaviate.classes.init import Auth

# Connect to Weaviate Cloud
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
)

print(client.is_ready())

True


In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("/content/RAG.pdf",extract_images=True)
pages=loader.load()

In [6]:
pages

[Document(metadata={'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2021-04-13T00:48:38+00:00', 'author': '', 'keywords': '', 'moddate': '2021-04-13T00:48:38+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/content/RAG.pdf', 'total_pages': 19, 'page': 0, 'page_label': '1'}, page_content='Retrieval-Augmented Generation for\nKnowledge-Intensive NLP Tasks\nPatrick Lewis†‡, Ethan Perez⋆,\nAleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,\nMike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†\n†Facebook AI Research; ‡University College London; ⋆New York University;\nplewis@fb.com\nAbstract\nLarge pre-trained language models have been shown to store factual knowledge\nin their parameters, and achieve state-of-the-art results when ﬁne-tuned on down-\nstream NLP tasks. Ho

In [25]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100)
docs = text_splitter.split_documents(pages)

In [26]:
docs

[Document(metadata={'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2021-04-13T00:48:38+00:00', 'author': '', 'keywords': '', 'moddate': '2021-04-13T00:48:38+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/content/RAG.pdf', 'total_pages': 19, 'page': 0, 'page_label': '1'}, page_content='Retrieval-Augmented Generation for\nKnowledge-Intensive NLP Tasks\nPatrick Lewis†‡, Ethan Perez⋆,\nAleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,\nMike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†\n†Facebook AI Research; ‡University College London; ⋆New York University;\nplewis@fb.com\nAbstract\nLarge pre-trained language models have been shown to store factual knowledge'),
 Document(metadata={'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '

In [9]:
# !pip install langchain-weaviate

In [27]:
for d in docs:
    d.metadata = {k.replace(".", "_"): v for k, v in d.metadata.items()}

In [28]:
from langchain_weaviate.vectorstores import WeaviateVectorStore

vector_db = WeaviateVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    client=client,
    index_name="Documents",
)

In [37]:
print(vector_db.similarity_search("Explain about rag", k=3)[0].page_content)

paper, as well as HuggingFace for their help in open-sourcing code to run RAG models. The authors
would also like to thank Kyunghyun Cho and Sewon Min for productive discussions and advice. EP
thanks supports from the NSF Graduate Research Fellowship. PL is supported by the FAIR PhD
program.
References
[1] Payal Bajaj, Daniel Campos, Nick Craswell, Li Deng, Jianfeng Gao, Xiaodong Liu, Rangan
Majumder, Andrew McNamara, Bhaskar Mitra, Tri Nguyen, Mir Rosenberg, Xia Song, Alina


In [36]:
print(vector_db.similarity_search("Explain about rag", k=3)[1].page_content)

and TriviaQA. RAG compares favourably to the DPR QA system, which uses a BERT-based “cross-
encoder” to re-rank documents, along with an extractive reader. RAG demonstrates that neither a
re-ranker nor extractive reader is necessary for state-of-the-art performance.
There are several advantages to generating answers even when it is possible to extract them. Docu-
ments with clues about the answer but do not contain the answer verbatim can still contribute towards


In [35]:
print(vector_db.similarity_search("Explain about rag", k=3)[2].page_content)

1Code to run experiments with RAG has been open-sourced as part of the HuggingFace Transform-
ers Library [66] and can be found at https://github.com/huggingface/transformers/blob/master/
examples/rag/. An interactive demo of RAG models can be found at https://huggingface.co/rag/
2


In [38]:
from langchain_core.prompts import ChatPromptTemplate

template = """
You are a helpful AI assistant. Use ONLY the information provided in the context to answer the question.

If the answer is not present in the context, say:
"I cannot find this information in the provided documents."

--- CONTEXT ---
{context}
--- END CONTEXT ---

QUESTION:
{question}

Answer clearly and concisely:
"""

prompt = ChatPromptTemplate.from_template(template)

In [16]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nYou are a helpful AI assistant. Use ONLY the information provided in the context to answer the question.\n\nIf the answer is not present in the context, say:\n"I cannot find this information in the provided documents."\n\n--- CONTEXT ---\n{context}\n--- END CONTEXT ---\n\nQUESTION:\n{question}\n\nAnswer clearly and concisely:\n'), additional_kwargs={})])

In [17]:
# !pip install langchain_google_genai

In [18]:
from google.colab import userdata
google_api_key = userdata.get('GOOGLE_API_KEY')

In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    google_api_key=google_api_key,
)

In [21]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

retriever = vector_db.as_retriever()

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [39]:
rag_chain.invoke("What is RAG system?")

'RAG is a system that combines the generation flexibility of "closed-book" (parametric only) approaches and the performance of "open-book" retrieval-based approaches. It achieves strong results without expensive, specialized “salient span masking” pre-training. RAG\'s retriever is initialized using DPR’s retriever, which uses retrieval supervision on Natural Questions and TriviaQA. RAG demonstrates that neither a re-ranker nor an extractive reader is necessary for state-of-the-art performance in a QA system.'